# Joining and Appending DataFrames in PySpark HW Solutions

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [2]:
# import findspark
# findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [3]:
import os

path = "Datasets/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".")  # separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path + filename, inferSchema=True, header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + " = df")

# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['subjects', 'subject_memberships', 'rooms', 'schedules', 'sections', 'courses', 'course_offerings', 'instructors', 'teachings', 'grade_distributions']


Now check the contents of a few of the dataframes that were read in above.

In [4]:
grade_distributions.limit(4).toPandas()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
**Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [4]:
step1 = rooms.join(sections, rooms.uuid == sections.room_uuid, how="left").select(
    [rooms.uuid, rooms.facility_code, sections.course_offering_uuid, "number"]
)
step1.limit(4).toPandas()

,uuid,facility_code,course_offering_uuid,number
0,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,611e5499-724c-3464-82f3-1a5bf29dd96c,307
1,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,43688423-905c-3455-bf40-a58adce537f7,3
2,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,43688423-905c-3455-bf40-a58adce537f7,3
3,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,871476fd-340f-303c-a7c6-44ca6ef6b84d,311


In [5]:
step2 = step1.join(
    course_offerings, step1.course_offering_uuid == course_offerings.uuid, how="left"
).select([rooms.uuid, rooms.facility_code, "number", "term_code", "name"])
step2.limit(4).toPandas()

,uuid,facility_code,number,term_code,name
0,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,307,1182,International Business
1,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,3,1174,Intro Managerial Accounting
2,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,3,1174,Intro Managerial Accounting
3,0feb8e9a-88b8-3d80-a13d-72c0c2fa9939,0140,311,1174,Business Law


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [6]:
step3 = step2.filter(step2.facility_code == "0469")
step3.limit(4).toPandas()

,uuid,facility_code,number,term_code,name
0,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,337,1082,Introduction to Philosophy
1,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,352,1082,Introduction to Philosophy
2,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,308,1102,Introduction to Philosophy
3,3c570ec9-1a27-3902-a16f-d1676890e63b,0469,307,1102,Introduction to Philosophy


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [7]:
step1 = (
    subjects.join(
        subject_memberships,
        subjects.code == subject_memberships.subject_code,
        how="inner",
    )
    .select(["name", "course_offering_uuid"])
    .withColumnRenamed("name", "subject_name")
)
step1.limit(4).toPandas()

,subject_name,course_offering_uuid
0,Chemical and Biological Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
1,Electrical and Computer Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
2,Engineering Mechanics and Astronautics,344b3ebe-da7e-314c-83ed-9425269695fd
3,Mechanical Engineering,344b3ebe-da7e-314c-83ed-9425269695fd


In [5]:
step2 = step1.join(
    sections, step1.course_offering_uuid == sections.course_offering_uuid, how="left"
).select(["subject_name", "room_uuid"])
step2.limit(4).toPandas()

,subject_name,room_uuid
0,Communication Arts,dd6118e0-7221-3b81-9b29-aad61f0ede54
1,Communication Arts,4e9c55b8-7c02-36c3-b907-3e832ea285e7
2,Communication Arts,4948c250-c6cf-3272-a497-f90962f3ba67
3,Communication Arts,698555af-8ec3-3b85-8e67-bd7fc53aba80


In [20]:
# I added a filter to make this a little simpler
step3 = (
    step2.join(rooms, step2.room_uuid == rooms.uuid, how="left")
    .filter('facility_code IN("0140","0545","0469","0031")')
    .select(["subject_name", "facility_code", "room_code"])
)
step3.limit(4).toPandas()

,subject_name,facility_code,room_code
0,Communication Arts,0140,1070
1,Communication Arts,0545,4028
2,Communication Arts,0545,4008
3,Communication Arts,0469,2241


In [21]:
# Option 1: Group by facility code and do a count
step3.groupBy("facility_code", "subject_name").count().orderBy("facility_code").show(
    10, False
)  # False prevents truncation of column content

+-------------+------------------------------------------------+-----+
|facility_code|subject_name                                    |count|
+-------------+------------------------------------------------+-----+
|0031         |Curriculum and Instruction                      |36   |
|0031         |Statistics                                      |6    |
|0031         |Philosophy                                      |1    |
|0031         |Communication Sciences and Disorders            |5    |
|0031         |Interdisciplinary Courses (L&S)                 |1    |
|0031         |Interdisciplinary Courses (Engineering)         |1    |
|0031         |Biochemistry                                    |2    |
|0031         |Electrical and Computer Engineering             |2    |
|0031         |History of Science                              |1    |
|0031         |Environmental Studies - Gaylord Nelson Institute|1    |
+-------------+------------------------------------------------+-----+
only s

In [24]:
# Option 2: Groupby subject name and pivot the facility code
# to see each facility side by side within each subject
step3.groupBy("subject_name").pivot("facility_code").count().show(10, False)

+---------------------------------+----+----+----+----+
|subject_name                     |0031|0140|0469|0545|
+---------------------------------+----+----+----+----+
|Asian American Studies           |null|6   |245 |24  |
|Religious Studies                |null|32  |282 |43  |
|HEBREW                           |null|7   |4   |null|
|Botany                           |null|4   |24  |2   |
|Urban and Regional Planning      |null|208 |17  |6   |
|Nutritional Sciences             |6   |null|null|28  |
|Kinesiology                      |1549|null|null|null|
|Art Education (Department of Art)|null|null|145 |null|
|Philosophy                       |1   |80  |1296|73  |
|Microbiology                     |null|null|1   |null|
+---------------------------------+----+----+----+----+
only showing top 10 rows



## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [5]:
grade_distributions.limit(4).toPandas()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [9]:
course_offerings.limit(4).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog


In [12]:
step1 = grade_distributions.groupBy("course_offering_uuid").sum("f_count")
step1.limit(4).toPandas()

,course_offering_uuid,sum(f_count)
0,c939dd5e-43ba-3266-8f74-573f629de75b,0
1,75fdf27b-0e52-3544-96eb-d594a5ad969f,0
2,cfeba94d-8e0e-320b-a904-ea2c2a31c000,7
3,ceabe145-78e5-33c2-9b68-3a1eea9c2864,4


In [22]:
step2 = (
    step1.join(
        course_offerings,
        step1.course_offering_uuid == course_offerings.uuid,
        how="left",
    )
    .select(["name", "sum(f_count)"])
    .orderBy("sum(f_count)")
)
step2.toPandas().tail(5)

,name,sum(f_count)
80166,Calculus&Analytic Geometry,63
80167,Calculus&Analytic Geometry 1,64
80168,Calculus&Analytic Geometry,67
80169,Animal Biology,70
80170,Calc--Functns of Variables,72


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [8]:
values = [
    ("Monkey", 10),
    ("Monkay", 36),
    ("Mnky", 123),
    ("Elephant", 48),
    ("Elefant", 16),
    ("Ellafant", 1),
    ("Hippopotamus", 48),
    ("Hipopotamus", 16),
    ("Hippo", 1),
]
zoo = spark.createDataFrame(values, ["Animal", "age"])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [14]:
# With the levenshtein distance!
from pyspark.sql.functions import *
from pyspark.sql.types import *

# First we create a dataframe with the 3 options we want to choose from
options = spark.createDataFrame(["Monkey", "Elephant", "Hippopotamus"], StringType())
options.show()

+------------+
|       value|
+------------+
|      Monkey|
|    Elephant|
|Hippopotamus|
+------------+



In [16]:
# And then we join the two dataframes together with a condition >5
results = zoo.join(options, levenshtein(zoo["Animal"], options["value"]) < 5, "left")
results.show()

+------------+---+------------+
|      Animal|age|       value|
+------------+---+------------+
|      Monkey| 10|      Monkey|
|      Monkay| 36|      Monkey|
|        Mnky|123|      Monkey|
|    Elephant| 48|    Elephant|
|     Elefant| 16|    Elephant|
|    Ellafant|  1|    Elephant|
|Hippopotamus| 48|Hippopotamus|
| Hipopotamus| 16|Hippopotamus|
|       Hippo|  1|        null|
+------------+---+------------+



So we can see here that all of our values were correctly identified except for "Hippo" which was just way too different from "Hippopotamus" to get correctly identified. So this solution won't work for EVERY case, but we can see here that it did a great job correcting simple gramatical errors. 

### Great job! 